# Image Filtering: Spatial Methods

In [ ]:
%load_ext cython

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage.filters import rank
from skimage.util import img_as_ubyte

from scipy.ndimage import convolve

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
I1 = io.imread("../../images/umbc.png", as_gray=True)
I1 = img_as_ubyte(I1) # needed for skimage.filters.rank

#io.imshow(I1)

In [ ]:
# add salt-and-pepper noise
rng = np.random.default_rng(472)
noise = rng.random(I1.shape)

In = np.copy(I1)
In[noise > 0.95] = 255
In[noise < 0.05] = 0

#io.imshow(In)

## Non-Linear Filter: Kernel Masks (Footprints)

In [ ]:
def box(N, M=None):
    if M == None:
        M = N
        
    h = np.ones((N,M))
    return h

## Non-Linear Filter: Alpha-Trimmed Mean

In [ ]:
%%cython
import numpy as np  # this cell is standalone

cpdef alphamean_filter(I, kernel, float alpha):
    Ip = np.empty_like(I)
    
    cdef int h1 = kernel.shape[0], h0 = h1 // 2;
    cdef int w1 = kernel.shape[1], w0 = w1 // 2;
    
    cdef int M, p
    cdef int S, k
    
    Iz = np.pad(I, ((h0,h0),(w0,w0)), mode='reflect')
    
    for u in np.arange(0,I.shape[0]):
        for v in np.arange(0,I.shape[1]):
            values = Iz[u:u+h1, v:v+w1]  # slicing
            values = np.extract(0 < kernel, values)
            values = np.sort(values)
            
            M = values.size
            #p = np.min([alpha*M, M/2])
            #p = np.uint32(np.around(p))
            p = int(alpha*M+0.5)
            if p > M//2:
                p = M//2
            
            #Ip[u][v] = np.mean(values[p:M-p])
            S = 0
            for k in range(p,M-p):
                S += values[k]
            Ip[u][v] = int(S/(M-2*p)+0.5)
        
    return Ip

In [ ]:
%%time
I3 = alphamean_filter(In, box(3), 0.45)
io.imshow(I3)